In [6]:
import numpy as np
import pickle as pkl
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from vectorrepr.datasets.timeseries import TimeSeriesDataset
from vectorrepr.sampler import ConfigurableSampler
from vectorrepr.models import TimeSeriesTransformerEmbedding
from vectorrepr.trainer.contrastive.pairwise_train import train

##### 加载数据集

In [7]:
dataset = TimeSeriesDataset(
    "../data/processed/stock500_2023_2024_ts.parquet",
    time_idx="DateIdx",
    group_ids="Ticker",
    feature_columns=["Open", "High", "Low", "Close", "Volume", "Adj Close"],
    na_handling=-1,
    input_steps=30,
    predict_steps=5,
)
print(len(dataset))
print("Shape of sample: ", dataset[0][0].shape)
print("Shape of label: ", dataset[0][1].shape)
sampled_candidates = pkl.load(open("../data/processed/stock500_2023_2024_sampled_candidates_v2.pkl", 'rb'))
anchors = [x[0] for x in sampled_candidates]
candidates = [x[1] for x in sampled_candidates]
scores = [x[2] for x in sampled_candidates]
sampler = ConfigurableSampler(
    dataset, anchors, candidates, scores
)

loader = DataLoader(sampler, batch_size=2, shuffle=True)

261560
Shape of sample:  (30, 6)
Shape of label:  (5, 6)


In [10]:
for i, (anchor_sample, candidate_sample, score) in enumerate(loader):
    print(anchor_sample.shape)
    print(candidate_sample.shape)
    print(score.shape)
    break
# 初始化模型、损失函数和优化器
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = TimeSeriesTransformerEmbedding(6, 512, 8, 6).to(device)
# criterion = nn.MSELoss(reduction="none")
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# train(loader, model, criterion, epochs=10, learning_rate=0.0001)

torch.Size([2, 6])
torch.Size([2, 6])
torch.Size([2])
